In [15]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from soilgasflux_fcs import multiprocess_raw_data, json_reader, fcs
from soilgasflux_fcs import synthetic_postProcessing
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# /Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-24
# data_folder = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/gals_20250829/sgf_07_pt02/data/2025-08-23')
data_folder = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-23')
a = json_reader.Initializer(folderPath=data_folder)
df = a.prepare_rawdata()

In [40]:
df.dtypes

datetime_utc               object
si_temperature            float64
bmp_temperature           float64
si_humidity               float64
datetime           datetime64[ns]
k30_co2                     int64
bmp_pressure              float64
id                         object
timedelta                   int64
dtype: object

In [41]:
# fig, ax = plt.subplots()
dfs = []
for i in df['id'].unique():
    df_id = df.loc[df['id']==i]
    df_id = df_id.sort_values(by='timedelta')

    df_id.set_index('datetime', inplace=True)
    df_id = df_id.asfreq('S')

    df_id_interpolated = df_id.interpolate(method='linear')
    df_id_interpolated['id'] = df_id_interpolated['id'].fillna(method='ffill')
    df_id_interpolated.reset_index(inplace=True)
    dfs.append(df_id_interpolated)

    # print(df_id_interpolated)
    # break
df_interpolated = pd.concat(dfs)

In [42]:
for i in df_interpolated['id'].unique():
    df_id = df_interpolated.loc[df_interpolated['id']==i]
    

In [43]:
df_interpolated

,datetime,datetime_utc,si_temperature,bmp_temperature,si_humidity,k30_co2,bmp_pressure,id,timedelta
0,2025-08-23 00:00:32,2025-08-23 00:00:32,18.86268,21.180,75.05469,517.0,96969.27,2025-08-23_00-00-32,0.0
1,2025-08-23 00:00:33,NaN,18.86268,21.195,75.14624,517.5,96968.97,2025-08-23_00-00-32,1.0
2,2025-08-23 00:00:34,2025-08-23 00:00:34,18.86268,21.210,75.23779,518.0,96968.67,2025-08-23_00-00-32,2.0
3,2025-08-23 00:00:35,2025-08-23 00:00:35,18.86268,21.250,75.48193,519.0,96968.59,2025-08-23_00-00-32,3.0
4,2025-08-23 00:00:36,2025-08-23 00:00:36,18.85195,21.280,75.69556,519.0,96968.75,2025-08-23_00-00-32,4.0
...,...,...,...,...,...,...,...,...,...
135,2025-08-24 00:01:58,2025-08-24 00:01:58,14.29379,17.840,91.21375,675.0,97139.19,2025-08-23_23-59-43,135.0
136,2025-08-24 00:01:59,2025-08-24 00:01:59,14.27234,17.850,91.21375,675.0,97138.84,2025-08-23_23-59-43,136.0
137,2025-08-24 00:02:00,2025-08-24 00:02:00,14.27234,17.860,91.21375,677.0,97138.95,2025-08-23_23-59-43,137.0
138,2025-08-24 00:02:01,2025-08-24 00:02:01,14.27234,17.870,91.21375,678.0,97137.18,2025-08-23_23-59-43,138.0


In [44]:
df_interpolated.groupby(by='id')['timedelta'].max().min(),df_interpolated.groupby(by='id')['timedelta'].max().max()


(139.0, 141.0)

In [45]:
df_interpolated.loc[df_interpolated['timedelta']>df_interpolated.groupby(by='id')['timedelta'].max().min()] = np.nan

In [46]:
df_interpolated.dropna(inplace=True)

In [47]:
df_interpolated

,datetime,datetime_utc,si_temperature,bmp_temperature,si_humidity,k30_co2,bmp_pressure,id,timedelta
0,2025-08-23 00:00:32,2025-08-23 00:00:32,18.86268,21.18,75.05469,517.0,96969.27,2025-08-23_00-00-32,0.0
2,2025-08-23 00:00:34,2025-08-23 00:00:34,18.86268,21.21,75.23779,518.0,96968.67,2025-08-23_00-00-32,2.0
3,2025-08-23 00:00:35,2025-08-23 00:00:35,18.86268,21.25,75.48193,519.0,96968.59,2025-08-23_00-00-32,3.0
4,2025-08-23 00:00:36,2025-08-23 00:00:36,18.85195,21.28,75.69556,519.0,96968.75,2025-08-23_00-00-32,4.0
5,2025-08-23 00:00:37,2025-08-23 00:00:37,18.87340,21.31,75.97021,520.0,96967.47,2025-08-23_00-00-32,5.0
...,...,...,...,...,...,...,...,...,...
135,2025-08-24 00:01:58,2025-08-24 00:01:58,14.29379,17.84,91.21375,675.0,97139.19,2025-08-23_23-59-43,135.0
136,2025-08-24 00:01:59,2025-08-24 00:01:59,14.27234,17.85,91.21375,675.0,97138.84,2025-08-23_23-59-43,136.0
137,2025-08-24 00:02:00,2025-08-24 00:02:00,14.27234,17.86,91.21375,677.0,97138.95,2025-08-23_23-59-43,137.0
138,2025-08-24 00:02:01,2025-08-24 00:02:01,14.27234,17.87,91.21375,678.0,97137.18,2025-08-23_23-59-43,138.0


In [48]:
a = multiprocess_raw_data.Multiprocessor()

# b = a.run(df=df, chamber_id='test01_20250823', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results')
b = a.run(df=df_interpolated, chamber_id='test01_20250823', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/results')

Multiprocessing started
CPU core count: 8
Date: 2025-08-23


/Users/alexnaokiasatokobayashi/opt/anaconda3/envs/Peat/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/alexnaokiasatokobayashi/opt/anaconda3/envs/Peat/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/alexnaokiasatokobayashi/opt/anaconda3/envs/Peat/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/alexnaokiasatokobayashi/opt/anaconda3/envs/Peat/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from panda

Processing ID: 2025-08-23_01-02-32
slice indices must be integers or None or have an __index__ method
ERROR HM ####
'NoneType' object is not subscriptable
slice indices must be integers or None or have an __index__ method
'NoneType' object has no attribute 'best_values'
ERROR LINEAR ####
'NoneType' object is not subscriptable
slice indices must be integers or None or have an __index__ method
ERROR HM ####
'NoneType' object is not subscriptable
slice indices must be integers or None or have an __index__ method
'NoneType' object has no attribute 'best_values'
ERROR LINEAR ####
'NoneType' object is not subscriptable
slice indices must be integers or None or have an __index__ method
ERROR HM ####
'NoneType' object is not subscriptable
slice indices must be integers or None or have an __index__ method
'NoneType' object has no attribute 'best_values'
ERROR LINEAR ####
'NoneType' object is not subscriptable
slice indices must be integers or None or have an __index__ method
ERROR HM ####
'None

ValueError: Variable 'dcdt(HM)': Could not convert tuple of form (dims, data[, attrs, encoding]): (['time', 'cutoff', 'deadband'], array([array([[nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan]]),
       array([[nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan]]),
       array([[nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, nan, nan],
              [nan, nan, nan, nan, na